# `sanic.app`

最基本的构建块是 {class}`sanic.app.Sanic` 实例。虽然这不是必需的，但通常的做法是在名为 `server.py` 的文件中实例化这个类。

```python
# /path/to/server.py

from sanic import Sanic

app = Sanic("MyHelloWorldApp")
```

## 应用上下文

大多数应用程序都需要在代码库的不同部分之间共享/重用数据或对象。Sanic 通过在应用程序实例上提供 `ctx` 对象来帮助实现这一点。这是供开发人员附加任何应在应用程序的整个生命周期内存在的对象或数据的自由空间。

最常见的模式是将数据库实例附加到应用程序上：

```python
from sanic import Sanic
app = Sanic("MyApp")
app.ctx.db = Database()
```

虽然前面的示例将起作用并具有说明性，但通常认为最佳实践是在两个应用程序启动[监听器](https://sanic.dev/en/guide/basics/listeners.html)之一中附加对象。

```python
app = Sanic("MyApp")

@app.before_server_start
async def attach_db(app, loop):
    app.ctx.db = Database()
```

## app 注册表

当你实例化 `Sanic` 实例时，稍后可以从 `Sanic` 应用程序注册表中检索它。例如，如果你需要从其他无法访问的位置访问你的 `Sanic` 实例，这可能会很有用。

```python
# ./path/to/server.py
from sanic import Sanic

app = Sanic("my_awesome_server")

# ./path/to/somewhere_else.py
from sanic import Sanic

app = Sanic.get_app("my_awesome_server")
```

如果你在不存在的应用程序上调用 `Sanic.get_app("non-existing")`，它默认会引发 {class}`sanic.exceptions.SanicException`。你也可以强制该方法返回具有该名称的新 `Sanic` 实例。

In [3]:
app = Sanic.get_app(
    "non-existing",
    force_create=True,
)

如果只注册了一个 `Sanic` 实例，那么调用 `Sanic.get_app()` 而不带参数将返回该实例。

```python
Sanic("My only app")

app = Sanic.get_app()
```

## 应用配置

在 Sanic 实例的 `config` 属性中持有配置。可以使用点表示法或像字典一样修改配置。

```python
app = Sanic('myapp')

app.config.DB_NAME = 'appdb'
app.config['DB_USER'] = 'appuser'

db_settings = {
    'DB_HOST': 'localhost',
    'DB_NAME': 'appdb',
    'DB_USER': 'appuser'
}
app.config.update(db_settings)
```

````{note}
配置键应该是大写的。但这主要是出于约定，大多数情况下小写也可以工作。
```python
app.config.GOOD = "yay!"
app.config.bad = "boo"
```
````

## 工厂模式

这些文档中的许多示例将展示在名为 `server.py` 的文件中实例化 `sanic.app.Sanic` 实例，位于“全局范围”（即不在函数内部）。这是非常简单的“hello world”风格应用程序的常见模式，但使用工厂模式往往更有利。

“工厂”只是返回你想要使用的对象实例的函数。这允许你抽象化对象的实例化过程，但也可能会使隔离应用程序实例变得更容易。

超级简单的工厂模式可能看起来像这样：

```python
# ./path/to/server.py
from sanic import Sanic
from .path.to.config import MyConfig
from .path.to.some.blueprint import bp


def create_app(config=MyConfig) -> Sanic:
    app = Sanic("MyApp", config=config)
    app.blueprint(bp)
    return app
```

当我们稍后运行 Sanic 时，你将了解到 Sanic CLI 可以检测到这种模式并使用它来运行你的应用程序。

```bash
sanic path.to.server:create_app
```